In [127]:
import pandas as pd
from itertools import product

In [128]:
logistic_df = pd.read_pickle('results/logistic-2024-Jan-01-04-00-06/results.pkl')
hypercube_df = pd.read_pickle('results/hypercube-2024-Jan-01-04-00-12/results.pkl')

In [130]:
logistic_df.columns.names = ['method', 'metric', 'statistic']
hypercube_df.columns.names = ['method', 'metric', 'statistic']

In [131]:
def by_dataset_by_model(df, dataset):
    unstacked = df.unstack().reset_index(drop=True)
    unstacked.index = [dataset]
    unstacked.index.name = 'dataset'
    return unstacked.stack()

In [132]:
combined = pd.concat([by_dataset_by_model(logistic_df, 'logistic'), by_dataset_by_model(hypercube_df, 'hypercube')])

In [135]:
def combine_mean_std(row):
    return f'{row["mean"]: .3f} ± {row["std"]: .3f}'

table = []
for comb in product(['cp', 'cp_adjusted'], ['cvg', 'size']):
    aggr = combined.loc[:, comb].apply(combine_mean_std, axis=1)
    aggr.name = comb
    table.append(aggr)

table = pd.concat(table, axis=1)

In [137]:
col_d = dict(zip(table.columns.levels[0], ['CP', 'CP Adjusted']))
col_d.update(zip(table.columns.levels[1], ['Coverage', 'Size']))
row_d = dict(zip(table.index.levels[0], ['Hypercube', 'Logistic']))
row_d.update(zip(table.index.levels[1], ['GBT', 'LR', 'MLP', 'RF']))
table = table.rename(columns=col_d).rename(row_d)

In [138]:
table.index.names = ['Dataset', 'Model']

In [145]:
latex = table.to_latex(formatters={'Dataset': lambda s: f'\\texttt{s}', 'Model': lambda s: f'\\texttt{s}'})
print(latex)

\begin{tabular}{llllll}
\toprule
          &    & \multicolumn{2}{l}{CP} & \multicolumn{2}{l}{CP Adjusted} \\
          &    &         Coverage &             Size &         Coverage &             Size \\
Dataset & Model &                  &                  &                  &                  \\
\midrule
Logistic & GBT &   0.968 ±  0.005 &   2.998 ±  0.120 &   0.915 ±  0.005 &   2.228 ±  0.174 \\
          & LR &   0.977 ±  0.005 &   2.833 ±  0.120 &   0.915 ±  0.006 &   2.002 ±  0.163 \\
          & MLP &   0.974 ±  0.005 &   2.910 ±  0.135 &   0.916 ±  0.005 &   2.062 ±  0.194 \\
          & RF &   0.964 ±  0.006 &   3.119 ±  0.131 &   0.916 ±  0.006 &   2.346 ±  0.203 \\
Hypercube & GBT &   0.983 ±  0.003 &   2.854 ±  0.051 &   0.915 ±  0.005 &   1.738 ±  0.089 \\
          & LR &   0.951 ±  0.008 &   3.495 ±  0.149 &   0.917 ±  0.005 &   3.050 ±  0.247 \\
          & MLP &   0.989 ±  0.002 &   2.707 ±  0.051 &   0.915 ±  0.005 &   1.493 ±  0.067 \\
          & RF &   0.982 ±  0.0